In [0]:
###############################################
#          Author: Daniel Cifuentes           #
#          Last update: 22/06/2025            #
###############################################

# Definir nombres de catalog, schema, volume 

catalog_name        = "integration_catalog"
schema_name_raw     = "raw_files"
volume_name_raw     = "files"
volume_name_tmp     = "tmp"
schema_name_tables  = "raw_tables"
workspace_folder       = "lab4"

In [0]:
# Copia los archivos del repositorio GitHub a local_tmp_path
import requests
import os

# ✅ Tus variables
catalog_name = "integration_catalog"
schema_name_raw = "raw_files"
volume_name_tmp = "tmp"

# 🔗 Repositorio GitHub (cambia si quieres usar otro)
github_user = "dancifuentesperu"
repo_name = "databricks_files"
branch = "main"

# 🔧 Construir paths
volume_path = f"/Volumes/{catalog_name}/{schema_name_raw}/{volume_name_tmp}"
dbfs_path = f"dbfs:{volume_path}"

# 🌐 GitHub API para listar archivos del repo
api_url = f"https://api.github.com/repos/{github_user}/{repo_name}/git/trees/{branch}?recursive=1"

# 📥 Obtener lista de archivos
response = requests.get(api_url)
response.raise_for_status()
tree = response.json().get("tree", [])

# 🔁 Descargar y copiar cada archivo al Volume UC
for item in tree:
    if item["type"] == "blob":  # Asegura que es un archivo
        file_path = item["path"]
        print(f"📄 Descargando: {file_path}")
        
        # Construir URL RAW
        raw_url = f"https://raw.githubusercontent.com/{github_user}/{repo_name}/{branch}/{file_path}"
        
        # Descargar contenido del archivo
        r = requests.get(raw_url)
        if r.status_code != 200:
            print(f"❌ Error al descargar: {raw_url}")
            continue

        # Guardar temporalmente en /Volumes/{catalog_name}/{schema_name_raw}/{volume_name_tmp}
        tmp_filename = os.path.basename(file_path)
        local_tmp_path = f"/Volumes/{catalog_name}/{schema_name_raw}/{volume_name_tmp}/{tmp_filename}"
        with open(local_tmp_path, "wb") as f:
            f.write(r.content)

        # Subir al Volume UC (manteniendo la ruta del archivo)


📄 Descargando: 0.Catalogs.py
📄 Descargando: 1.Files.py
📄 Descargando: 2.Tables.py
📄 Descargando: README.md
📄 Descargando: flights.parquet
📄 Descargando: titanic.parquet


In [0]:
# Variables
catalog_name        = "integration_catalog"
schema_name_raw     = "raw_files"
volume_name_raw     = "tmp"       # Volumen origen
volume_name_target  = "files"   # Volumen destino (ajusta si es otro)

# Rutas de origen y destino
source_volume_path = f"/Volumes/{catalog_name}/{schema_name_raw}/{volume_name_raw}/"
target_volume_path = f"/Volumes/{catalog_name}/{schema_name_raw}/{volume_name_target}/"

# Listar archivos y copiar solo los .parquet
files = dbutils.fs.ls(source_volume_path)

for file in files:
    if file.name.endswith(".parquet"):
        src = f"{source_volume_path}{file.name}"
        dst = f"{target_volume_path}{file.name}"
        print(f"Copiando {file.name}...")
        dbutils.fs.cp(src, dst)

Copiando flights.parquet...
Copiando titanic.parquet...
